<a href="https://colab.research.google.com/github/Dhanyabahadur/ML/blob/master/Lego%20bricks%20image%20classifier%20question_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setting up Environment**

In [1]:
!pip install PyDrive                                                                                                                                                                                                                                                              

**Importing Libraries**

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

*Creating a Drive Variable to access Google Drive*

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

*Downloading the train.zip file, using ID of the file uploaded on Google Drive*

In [0]:
# Train.zip
download = drive.CreateFile({'id': '1nPTyAeeEjBdkEDpUYrVJRJ2aXaYxoa6h'})

In [0]:
# Train.zip
folder_id = '1nPTyAeeEjBdkEDpUYrVJRJ2aXaYxoa6h'
lister = drive.ListFile({'q': "'%s' in parents" % folder_id}).GetList()
for item in lister:
    print(item['title'])
    # this should tell you which mimetype the file you're trying to download 
    # has. 
    print('title: %s, mimeType: %s' % (item['title'], item['mimeType']))
    mimetypes = {



        # Drive Document files as PDF
        'application/vnd.google-apps.document': 'application/pdf',

        # Drive Sheets files as MS Excel files.
        'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',

        # Drive document as folder
        'application/vnd.google-apps.folder': 'application/folder',

        #Drive document as image
        'application/vnd.google-apps.photo': 'application/png',

    # see https://developers.google.com/drive/v3/web/mime-types
    }
    download_mimetype = None
    if item['mimeType'] in mimetypes:
        download_mimetype = mimetypes[item['mimeType']]
        item.GetContentFile(item['title'], mimetype=download_mimetype)

        #item.GetContentFile(item['title'], mimetype=download_mimetype)
    else: 
        item.GetContentFile(item['title'])

*Now we will download this file and unzip it*

In [6]:
download.GetContentFile('Train.zip')
!unzip  -o Train.zip

Archive:  Train.zip
   creating: train/
  inflating: train/3218.png          
  inflating: train/2093.png          
  inflating: train/1653.png          
  inflating: train/2817.png          
  inflating: train/3588.png          
  inflating: train/2720.png          
  inflating: train/1665.png          
  inflating: train/1777.png          
  inflating: train/1827.png          
  inflating: train/3235.png          
  inflating: train/667.png           
  inflating: train/3220.png          
  inflating: train/4066.png          
  inflating: train/4093.png          
  inflating: train/895.png           
  inflating: train/4392.png          
  inflating: train/1708.png          
  inflating: train/1915.png          
  inflating: train/2933.png          
  inflating: train/3789.png          
  inflating: train/2305.png          
  inflating: train/777.png           
  inflating: train/1211.png          
  inflating: train/2223.png          
  inflating: train/3414.png          
  inflatin

*Importing Required Libraries for Model Building* 

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import RMSprop
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


**Data Preprocessing**

In [0]:
# Train.csv
download = drive.CreateFile({'id': '1B8Aodo2QlZLtwnNp1GxcVuC2bg2Egjsn'})

In [0]:
# Train.csv
folder_id = '1B8Aodo2QlZLtwnNp1GxcVuC2bg2Egjsn'
lister = drive.ListFile({'q': "'%s' in parents" % folder_id}).GetList()
for item in lister:
    print(item['title'])
    # this should tell you which mimetype the file you're trying to download 
    # has. 
    
    print('title: %s, mimeType: %s' % (item['title'], item['mimeType']))
    mimetypes = {



        # Drive Document files as PDF
        'application/vnd.google-apps.document': 'application/pdf',

        # Drive Sheets files as MS Excel files.
        'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',

        # Drive document as folder
        'application/vnd.google-apps.folder': 'application/folder',

        #Drive document as image
        'application/vnd.google-apps.photo': 'application/png',

    # see https://developers.google.com/drive/v3/web/mime-types
    }
    download_mimetype = None
    if item['mimeType'] in mimetypes:
        download_mimetype = mimetypes[item['mimeType']]
        item.GetContentFile(item['title'], mimetype=download_mimetype)

        #item.GetContentFile(item['title'], mimetype=download_mimetype)
    else: 
        item.GetContentFile(item['title'])

In [0]:
download.GetContentFile('Train.csv')

In [0]:
train = pd.read_csv('Train.csv')

*We will read all the training images,store them in a list, and finally convert that list into a numpy array*

In [12]:
# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('train/'+train['name'][i], target_size=(28,28,1), grayscale = True)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

  0%|          | 0/4465 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 4465/4465 [00:05<00:00, 883.92it/s]


*This is a multi-class classification problem with 16 classes , so we will one-hot encode the target variable*

In [0]:
y=train['category'].values
y = to_categorical(y)

*Let's create a validation set from the train set*

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

**Defining the model structure**

*Creating a simple architecture of 2 convolutional layers, one dense hidden layer and an output layer*

In [15]:
prior = keras.applications.VGG16(
    include_top=False, 
    weights='imagenet',
    input_shape=(48, 48, 3)
)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

58892288/58889256 [==============================] - 5s 0us/step


*Compiling the model we have created*

In [0]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

*Using EarlyStopping Callback*

In [0]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

*training the model using training set images and validating it using the validation set.I have tried 10, 15, 25 and 35 epochs.But I found the result of each epochs similar , so I am using 10 here*

In [19]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 3125 samples, validate on 1340 samples
Epoch 1/10
3125/3125 [==============================] - 1s 369us/step - loss: 1.1729 - accuracy: 0.5446 - val_loss: 0.7926 - val_accuracy: 0.7351
Epoch 2/10
3125/3125 [==============================] - 1s 369us/step - loss: 0.9963 - accuracy: 0.6243 - val_loss: 0.6986 - val_accuracy: 0.7575
Epoch 3/10
3125/3125 [==============================] - 1s 362us/step - loss: 0.8354 - accuracy: 0.6832 - val_loss: 0.5915 - val_accuracy: 0.7858
Epoch 4/10
3125/3125 [==============================] - 1s 364us/step - loss: 0.7121 - accuracy: 0.7258 - val_loss: 0.5198 - val_accuracy: 0.7940
Epoch 5/10
3125/3125 [==============================] - 1s 358us/step - loss: 0.6604 - accuracy: 0.7507 - val_loss: 0.4782 - val_accuracy: 0.8201
Epoch 6/10
3125/3125 [==============================] - 1s 349us/step - loss: 0.5707 - accuracy: 0.7766 - val_loss: 0.4467 - val_accuracy: 0.8306
Epoch 7/10
3125/3125 [==============================] - 1s 351us/step - loss

**On Field!**

*Making Predictions*

*Repeating the initially followed steps with training data .Predicting their classes using the model.predict()_classes()function*

In [0]:
# test.zip
download = drive.CreateFile({'id': '1c6f6bE-QvkMFrqtPBVU6DnzBWBvjM4VC'})

In [0]:
# test.zip
folder_id = '1c6f6bE-QvkMFrqtPBVU6DnzBWBvjM4VC'
lister = drive.ListFile({'q': "'%s' in parents" % folder_id}).GetList()
for item in lister:
    print(item['title'])
    # this should tell you which mimetype the file you're trying to download 
    # has. 
    print('title: %s, mimeType: %s' % (item['title'], item['mimeType']))
    mimetypes = {



        # Drive Document files as PDF
        'application/vnd.google-apps.document': 'application/pdf',

        # Drive Sheets files as MS Excel files.
        'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',

        # Drive document as folder
        'application/vnd.google-apps.folder': 'application/folder',

        #Drive document as image
        'application/vnd.google-apps.photo': 'application/png',

    # see https://developers.google.com/drive/v3/web/mime-types
    }
    download_mimetype = None
    if item['mimeType'] in mimetypes:
        download_mimetype = mimetypes[item['mimeType']]
        item.GetContentFile(item['title'], mimetype=download_mimetype)

        #item.GetContentFile(item['title'], mimetype=download_mimetype)
    else: 
        item.GetContentFile(item['title'])

In [22]:
# test.zip
download.GetContentFile('test.zip')
!unzip -o test.zip

Archive:  test.zip
   creating: test/
  inflating: test/5784.png           
  inflating: test/5596.png           
  inflating: test/6098.png           
  inflating: test/6127.png           
  inflating: test/5235.png           
  inflating: test/6069.png           
  inflating: test/5573.png           
  inflating: test/4925.png           
  inflating: test/5473.png           
  inflating: test/5556.png           
  inflating: test/6221.png           
  inflating: test/4986.png           
  inflating: test/6370.png           
  inflating: test/5314.png           
  inflating: test/4724.png           
  inflating: test/6025.png           
  inflating: test/4998.png           
  inflating: test/5615.png           
  inflating: test/5225.png           
  inflating: test/5562.png           
  inflating: test/5570.png           
  inflating: test/5366.png           
  inflating: test/5265.png           
  inflating: test/5359.png           
  inflating: test/5145.png           
  inflating:

In [0]:
# Test.csv
download = drive.CreateFile({'id': '1PNWGHVkDZBicpfaUvkMzhwWiVETp42_t'})

download.GetContentFile('test.csv')

*Importing the test file i.e test.csv*

In [0]:
test = pd.read_csv('test.csv')

*Now we will read and store all the test images*

In [25]:
# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('test/'+test['name'][i], target_size=(28,28,1), grayscale = True)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

  0%|          | 0/1914 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 1914/1914 [00:02<00:00, 871.26it/s]


In [0]:
# making predictions
# sample_submission.csv
download = drive.CreateFile({'id': '1Cy8MCPGtzuqAty-BphT-8N0b3X67lyUS5IzR-SEiToY'})


#Test .csv
folder_id = '1Cy8MCPGtzuqAty-BphT-8N0b3X67lyUS5IzR-SEiToY'
lister = drive.ListFile({'q': "'%s' in parents" % folder_id}).GetList()
for item in lister:
    print(item['title'])
    # this should tell you which mimetype the file you're trying to download 
    # has. 
    print('title: %s, mimeType: %s' % (item['title'], item['mimeType']))
    mimetypes = {



        # Drive Document files as PDF
        'application/vnd.google-apps.document': 'application/pdf',

        # Drive Sheets files as MS Excel files.
        'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',

        # Drive document as folder
        'application/vnd.google-apps.folder': 'application/folder',

        #Drive document as image
        'application/vnd.google-apps.photo': 'application/png',

    # see https://developers.google.com/drive/v3/web/mime-types
    }
    download_mimetype = None
    if item['mimeType'] in mimetypes:
        download_mimetype = mimetypes[item['mimeType']]
        item.GetContentFile(item['title'], mimetype=download_mimetype)

        #item.GetContentFile(item['title'], mimetype=download_mimetype)
    else: 
        item.GetContentFile(item['title'])


download.GetContentFile('sample_submission.csv', mimetype='text/csv')

prediction = model.predict_classes(test)

**Finally SUBMISSION!**

In [0]:
# creating submission file
sample = pd.read_csv('sample_submission.csv')
sample['category'] = prediction
sample.to_csv('sample_cnn.csv', header = True , index = False)